<a id="top"></a>

# Homework 6: IMDb
## SQL

In [1]:
# Run this cell to set up your notebook

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
import sqlalchemy
from pathlib import Path

plt.style.use('fivethirtyeight') # Use plt.style.available to see more styles
sns.set()
sns.set_context("talk")
np.set_printoptions(threshold=5) # avoid printing out big matrices
%matplotlib inline
%load_ext sql

<br/><br/>

---
<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />

---

<a id='part-1'></a>
# Part 1: The IMDB (mini) Dataset


* **Caution: Be careful with large SQL queries!!** You may need to reboot your Jupyter notebook instance if it stops responding. **Use the LIMIT keyword** to avoid printing out 100k-sized tables (but remember to remove it).

In [2]:
# run this cell and the next one
engine = sqlalchemy.create_engine("sqlite:///data/imdbmini.db")
connection = engine.connect()

In [3]:
%sql sqlite:///data/imdbmini.db

<br/>

Let's take a look at the table schemas:

In [4]:
%%sql
-- just run this cell --
SELECT * FROM sqlite_master WHERE type='table';

 * sqlite:///data/imdbmini.db
Done.


type,name,tbl_name,rootpage,sql
table,Title,Title,2,"CREATE TABLE ""Title"" (""tconst"" INTEGER, ""titleType"" TEXT, ""primaryTitle"" TEXT, ""originalTitle"" TEXT, ""isAdult"" TEXT, ""startYear"" TEXT, ""endYear"" TEXT, ""runtimeMinutes"" TEXT, ""genres"" TEXT)"
table,Name,Name,12,"CREATE TABLE ""Name"" (""nconst"" INTEGER, ""primaryName"" TEXT, ""birthYear"" TEXT, ""deathYear"" TEXT, ""primaryProfession"" TEXT)"
table,Role,Role,70,"CREATE TABLE ""Role"" (tconst INTEGER,ordering TEXT,nconst INTEGER,category TEXT,job TEXT,characters TEXT)"
table,Rating,Rating,41,"CREATE TABLE ""Rating"" (tconst INTEGER,averageRating TEXT,numVotes TEXT)"


From running the above cell, we see the database has 4 tables: `Name`, `Role`, `Rating`, and `Title`.

<details>
    <summary>[<b>Click to Expand</b>] See descriptions of each table's schema.</summary>
    
**`Name`** – Contains the following information for names of people.
    
- nconst (text) - alphanumeric unique identifier of the name/person
- primaryName (text)– name by which the person is most often credited
- birthYear (integer) – in YYYY format
- deathYear (integer) – in YYYY format
    
    
**`Role`** – Contains the principal cast/crew for titles.
    
- tconst (text) - alphanumeric unique identifier of the title
- ordering (integer) – a number to uniquely identify rows for a given tconst
- nconst (text) - alphanumeric unique identifier of the name/person
- category (text) - the category of job that person was in
- characters (text) - the name of the character played if applicable, else '\\N'
    
**`Rating`** – Contains the IMDb rating and votes information for titles.
    
- tconst (integer) - alphanumeric unique identifier of the title
- averageRating (text) – weighted average of all the individual user ratings
- numVotes (text) - number of votes (i.e., ratings) the title has received
    
**`Title`** - Contains the following information for titles.
    
- tconst (text) - alphanumeric unique identifier of the title
- titleType (text) -  the type/format of the title
- primaryTitle (text) -  the more popular title / the title used by the filmmakers on promotional materials at the point of release
- isAdult (text) - 0: non-adult title; 1: adult title
- startYear (text) – represents the release year of a title.
- runtimeMinutes (integer)  – primary runtime of the title, in minutes
    
</details>

<br/><br/>
From the above descriptions, we can conclude the following:
* `Name.nconst` and `Title.tconst` are primary keys of the `Name` and `Title` tables, respectively.
* `Role.nconst` and `Role.tconst` are **foreign keys** that point to `Name.nconst` and `Title.tconst`, respectively.

<br/><br/>

<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />

## Question 1

### Question 1a
How far back does our data go? Does it only include recent data, or do we have information about older movies and movie stars as well? 

List the **10 oldest films** by `startYear` and then `primaryTitle` both in **ascending** order.  Do not include films where the `startYear` is `NULL`.  The output should contain the `startYear`, `primaryTitle`, and `titleType`.

Remember, you can create a `%%sql` cell **after** the grader cell as scratch work. Just be sure to copy the query back into the Python cell to run the autograder.

<!--
BEGIN QUESTION
name: q1a
points: 2
-->

In [5]:
query_q1a = """
...       # replace this with
...;      # your multi-line solution
"""


res_q1a = pd.read_sql(query_q1a, engine)
res_q1a

<br/>

<hr style="border: 1px solid #fdb515;" />

### Question 1b

Next, let's calculate the distribution of films by year. Write a query that returns the **total** films for each `startYear` in the `Title` table as `total`.  Keep in mind that some entries may not have a `startYear` listed -- you should filter those out.  Order your final results by the `startYear` in **ascending** order.

The first few records of the table should look like the following (but you should compute the entire table).


| |startYear|total|
|-----|------|-----|
|**0**|1902|1|
|**1**|1915|1|
|**2**|1920|1|
|**3**|1921|1|
|**4**|1922|1|
|...|...|...|


<!--
BEGIN QUESTION
name: q1b
points: 2
-->

In [10]:
query_q1b = """
...       # replace this with
...;      # your multi-line solution
"""


res_q1b = pd.read_sql(query_q1b, engine)
res_q1b

<br/>

The following should generate an interesting plot of the number of films that premiered each year. Notice there is a dip between the 1920s and late 1940s. Why might that be? *This question is rhetorical; you do not need to write your answer anywhere.*

In [14]:
# just run this cell
px.bar(res_q1b, x="startYear", y="total",
       title="Number of films premiered each year")

<br/><br/>

<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />

## Question 2

Who are the **top 10 most prolific movie actors**?

Define the term "movie actor" is defined as anyone with an `actor` or `actress` job category role in a `movie` title.

Your SQL query should output exactly two fields named `name` (the movie actor name) and `total` (the number of movies the movie actor appears in). Order the records by `total` in descending order, and break ties by ordering by `name` in ascending order.

Your result should look something like the following, but without `????`:

| | name | total |
|-----|-----|-----|
|**0**| ???? | 64 |
|**1**| ???? | 54 |
|**2**| ???? | 53 |
|**3**| ???? | 49 |
|**4**| ???? | 46 |
|**5**| ???? | 43 |
|**6**| ???? | 41 |
|**7**| ???? | 40 |
|**8**| ???? | 40 |
|**9**| ???? | 39 |

Some hints: 

* ***The query should take < 2 minutes to run.***
* If you want to include a non-aggregate field in the `SELECT` clause, it must also be included in the `GROUP BY` clause.
<!--* You can assume each movie actor only has one role per film. If you're not sure how this hint affects your query, ignore this hint.-->

<!--
BEGIN QUESTION
name: q2
points: 3
-->

In [15]:
# query_q2 = """
# ...       # replace this with
# ...;      # your multi-line solution
# """


res_q2 = pd.read_sql(query_q2, engine)
res_q2

In [20]:
sorted(list(res_q2['name']))

<br/><br/>

<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />

## Question 3: The `CASE` Keyword

The `Rating` table has the `numVotes` and the `averageRating` for each title. Which **_movies_** were **"big hits"**, defined as a movie with over 100,000 votes? Construct the following table:

| | isBigHit | total |
|-----|-----|-----|
|**0**| no | ???? |
|**1**| yes | ???? |

Where `????` is replaced with the correct values. The row with `no` should have the count for how many movies **are not** big hits, and the row with `yes` should have the count of how many movies **are** big hits.

* Rating.numVotes currently consists of string objects, use `CAST(Rating.numVotes AS int)` to convert them to integer.
* You will need to use  some type of `JOIN`.
* You may also consider using a `CASE WHEN ... IS ... THEN 'yes' ... ELSE ... END` statement. `CASE` statements are the SQL-equivalent of Python `if... elif... else` statements. To read up on `CASE`, take a look at the following links:
    - https://mode.com/sql-tutorial/sql-case/
    - https://www.w3schools.com/sql/sql_ref_case.asp

<!--
BEGIN QUESTION
name: q3
manual: false
points: 3
-->

In [21]:
query_q3 = """
...       # replace this with
...;      # your multi-line solution
"""


res_q3 = pd.read_sql(query_q3, engine)
res_q3

<br/><br/>

<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />

## Question 4

**How does film length relate to ratings?**  To answer this question we want to bin films by length and compute the average of the average ratings within each length bin. We will group movies by 10-minute increments -- that is, one bin for movies \[0, 10) minutes long, another for \[10, 20) minutes, another for \[20, 30) minutes, and so on. Use the following code snippet to help construct 10-minute bins: 

```
ROUND(runtimeMinutes / 10.0 + 0.5) * 10 AS runtimeBin
```

Construct a table containing the **`runtimeBin`**, the **average** of the **average ratings** (as `averageRating`), the **average number of votes** (as `averageNumVotes`), and the number of `titles` in that **runtimeBin** (as `total`).  Only include movies with **at least 10000 votes**.  Order the final results by the value of `runtimeBin`.

<!--
BEGIN QUESTION
name: q4
manual: false
points: 4
-->

In [26]:
query_q4 = """
...       # replace this with
...;      # your multi-line solution
"""


res_q4 = pd.read_sql(query_q4, engine)
res_q4.head()

<br/>

If your SQL query is correct you should get some interesting plots below.  This might explain why directors keep going a particular direction with film lengths.  

In [34]:
# just run this cell
px.bar(res_q4, x="runtimeBin", y="total",
       title="Distribution of Movie Runtimes")

In [35]:
# just run this cell
px.line(res_q4, x="runtimeBin", y="averageRating",
        title="Movie Ratings vs. Runtime")

In [36]:
px.line(res_q4, x="runtimeBin", y="averageNumVotes",
        title="Movie Number of Votes vs. Runtime")

<br/><br/>

<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />

## Question 5


Which **movie actors** have the highest average ratings across all the **movies** in which they star? Again, define "movie actor" as anyone with an `actor` or `actress` job category role in a `movie` title.

Construct a table consisting of the **movie actor's name**  (as `name`) and their **average actor rating** (as `actorRating`) computed by rescaling ratings for movies in which they had a role:

$$
\text{actorRating} = 
\frac{\sum_m \text{averageRating}[m] * \text{numVotes}[m]}{\sum_m \text{numVotes}[m]}
$$

Some notes:
* Note that if an actor/actress has multiple `role` listings for a film then that film will have a bigger impact in the overall average (this is desired).
* ***The query should take < 3 minutes to run.***
* Only consider ratings where there are **at least 1000** votes and only consider movie actors that have **at least 20 rated performances**. Present the movie actors with the **top 10** `actorRating` in descending order and break ties alphabetically using the movie actor's name.

The results should look something like this but without the `????`, and with higher rating precision.

| | name | actorRating |
|-----|-----|-----|
|**0**|????|8.4413...|
|**1**|????|8.2473...|
|**2**|????|8.1383...|
|**3**|????|8.1339...|
|**4**|????|8.0349...|
|**5**|????|7.9898...|
|**6**|????|7.9464...|
|**7**|????|7.9330...|
|**8**|????|7.9261...|
|**9**|????|7.8668...|


<!--
BEGIN QUESTION
name: q5
manual: false
points: 4
-->

In [37]:
query_q5 = """
...       # replace this with
...;      # your multi-line solution
"""


res_q5 = pd.read_sql(query_q5, engine)
res_q5

## Congratulations!

Congrats! You are finished with this homework assignment.

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. **Please save before exporting!**